In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cv2
import os
import seaborn as sns
import tensorflow as tf
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import Input, Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
from tensorflow_addons.losses import SigmoidFocalCrossEntropy
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, confusion_matrix
from tqdm.notebook import tqdm_notebook
import warnings 
warnings.filterwarnings('ignore')

# Helper function to load/preprocess images

In [2]:
def load_image(path):
    im = cv2.imread(path)
    im = cv2.cvtColor(im, cv2.COLOR_BGR2RGB)
    im = cv2.resize(im, (224,224))
    im = im/255.0
    
    return im

# Test image path and dataframe

In [3]:
path = './test'
test_df = pd.read_csv('test.csv')

# Model

In [5]:
model = tf.keras.models.load_model('sports.h5')

# Dictionary of labels created by keras generator

In [12]:
dict_label = {'Badminton': 0, 'Cricket': 1, 'Karate': 2, 'Soccer': 3, 'Swimming': 4, 'Tennis': 5, 'Wrestling': 6}
dict_pred = {v:k for k,v in dict_label.items()}

# Saving the predictions in submission.csv

In [14]:
predictions = []
for id_im in test_df['image_ID']:
    path_im = os.path.join(path,id_im)
    im = load_image(path_im)
    pred = model.predict(np.expand_dims(im,axis=0))
    pred = pred.argmax()
#     print(dict_pred[pred])
    predictions.append(dict_pred[pred])

In [21]:
sub = pd.DataFrame(columns = ['image_ID','label'])
sub['image_ID'] = test_df['image_ID']
sub['label'] = predictions

In [24]:
sub.to_csv('submission.csv',index=False)